
# Statistical Significance Analysis

This notebook performs statistical testing and uncertainty estimation on per-fold results across multiple ensemble models.

**Workflow**
1. Input: per-fold `R2` and `MSE` for each model.
2. Normality (per model on `R2`): Shapiro–Wilk.
3. Homogeneity of variances (across models on `R2`): Levene's test.
4. One-way ANOVA on `R2` to assess differences among models.
5. Post-hoc pairwise comparisons on `R2`: Tukey HSD, plus compact letter display (CLD).
6. Bootstrap confidence intervals for the mean `MSE` per model.
7. Outputs: summary table (`R2` mean/SD, CLD group, mean `MSE` and CI), ANOVA table, and figures (bar chart with CLD, bootstrap histograms).


In [ ]:

from __future__ import annotations

import os
from typing import List, Dict

import numpy as np
import pandas as pd

from scipy.stats import shapiro, levene
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)

# -------------------
# Configuration
# -------------------
# CSV with columns: model, fold, R2, MSE
METRICS_CSV = None  # e.g., "statsig_inputs/per_fold_metrics.csv"

# Model names must match the "model" column values
MODEL_NAMES = [
    "LightGBM+CatBoost",
    "GradientBoosting+XGBoost",
    "RandomForest+HistGB",
    "RandomForest+SVM",
    "MLP+KNN",
    "SVM+PolynomialReg",
    "Ridge+BayesianRidge",
]

# Bootstrap settings
CI_LEVEL = 0.95               # e.g., 0.90 / 0.95 / 0.99
ALPHA = 1.0 - CI_LEVEL
BOOTSTRAP_N = 10_000          # number of resamples

# Outputs
OUT_DIR = "statsig_outputs"
os.makedirs(OUT_DIR, exist_ok=True)



## Input schema
CSV must contain (one row per fold per model):
```
model,fold,R2,MSE
LightGBM+CatBoost,1,0.882,0.119
...
Ridge+BayesianRidge,5,0.640,0.362
```


In [ ]:

def load_metrics_from_csv(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    expected = {"model","fold","R2","MSE"}
    if not expected.issubset(df.columns):
        raise ValueError(f"CSV must contain columns: {expected}")
    df["model"] = pd.Categorical(df["model"], categories=MODEL_NAMES, ordered=True)
    df = df.sort_values(["model","fold"]).reset_index(drop=True)
    return df


In [ ]:

def compact_letter_display_from_tukey(tukey_res, model_order: List[str]) -> Dict[str,str]:
    """Build compact letter display (CLD) from Tukey HSD results.
    Models sharing a letter are not significantly different at the chosen alpha level.
    """
    names = tukey_res.groupsunique.tolist()
    reject = tukey_res.reject  # True => significant difference
    pairs = list(zip(tukey_res._multicomp.pairindices[0], tukey_res._multicomp.pairindices[1]))
    # adjacency for "not significantly different"
    adj = {n: set() for n in names}
    for (i,j), r in zip(pairs, reject):
        a, b = names[i], names[j]
        if not r:
            adj[a].add(b); adj[b].add(a)
    # greedy letter assignment
    groups = {m: set() for m in names}
    remaining = set(names)
    letter_idx = 0
    while remaining:
        letter = chr(ord('a') + letter_idx)
        this_group = set()
        for m in list(remaining):
            if all((n not in this_group) for n in adj[m] if n in remaining):
                this_group.add(m)
        for m in this_group:
            groups[m].add(letter)
        remaining -= this_group
        letter_idx += 1
    return {m: "".join(sorted(groups.get(m, set()))) for m in model_order}


In [ ]:

def run_statsig_pipeline(df_metrics: pd.DataFrame, ci_level: float = CI_LEVEL, bootstrap_n: int = BOOTSTRAP_N):
    assert 0 < ci_level < 1
    alpha = 1.0 - ci_level

    df = df_metrics.copy()
    df["model"] = pd.Categorical(df["model"], categories=MODEL_NAMES, ordered=True)

    # Shapiro–Wilk (R2 per model)
    shapiro_rows = []
    for m in MODEL_NAMES:
        vals = df.loc[df["model"] == m, "R2"].dropna().values
        if len(vals) < 3:
            W, p = np.nan, np.nan
        else:
            W, p = shapiro(vals)
        shapiro_rows.append({"model": m, "W": W, "p_value": p})
    shapiro_df = pd.DataFrame(shapiro_rows)

    # Levene (R2 across models)
    r2_groups = [df.loc[df["model"] == m, "R2"].dropna().values for m in MODEL_NAMES]
    lev_stat, lev_p = levene(*r2_groups)
    levene_df = pd.DataFrame([{"statistic": lev_stat, "p_value": lev_p, "k_groups": len(MODEL_NAMES)}])

    # One-way ANOVA (R2)
    anova_model = ols('R2 ~ C(model)', data=df).fit()
    anova_table = sm.stats.anova_lm(anova_model, typ=2).rename(
        index={"C(model)":"Between Groups","Residual":"Within Groups"}
    )
    anova_table = anova_table.rename(columns={"sum_sq":"SS","mean_sq":"MS","PR(>F)":"p_value"})
    total_row = pd.DataFrame([{
        "SS": anova_table["SS"].sum(),
        "df": anova_table["df"].sum(),
        "MS": np.nan, "F": np.nan, "p_value": np.nan
    }], index=["Total"])
    anova_out = pd.concat([anova_table[["SS","df","MS","F","p_value"]], total_row])

    # Tukey HSD (R2) + CLD
    tukey = pairwise_tukeyhsd(endog=df["R2"].values,
                              groups=df["model"].astype(str).values,
                              alpha=alpha)
    cld = compact_letter_display_from_tukey(tukey, MODEL_NAMES)

    # Bootstrap CI for mean MSE
    boot_rows = []
    rng = np.random.default_rng(42)
    low_q, high_q = 100*alpha/2, 100*(1-alpha/2)
    for m in MODEL_NAMES:
        mse_vals = df.loc[df["model"] == m, "MSE"].dropna().values
        if len(mse_vals) == 0:
            mean_mse = lo = hi = np.nan
        else:
            bs_means = [np.mean(rng.choice(mse_vals, size=len(mse_vals), replace=True))
                        for _ in range(bootstrap_n)]
            lo, hi = np.percentile(bs_means, [low_q, high_q])
            mean_mse = float(np.mean(mse_vals))
        boot_rows.append({"model": m, "Mean_MSE": mean_mse, f"CI{int(ci_level*100)}_low": lo, f"CI{int(ci_level*100)}_high": hi})
    boot_df = pd.DataFrame(boot_rows)

    # Summary table
    summary_rows = []
    for m in MODEL_NAMES:
        r2_vals = df.loc[df["model"] == m, "R2"].dropna().values
        mean_r2 = float(np.mean(r2_vals)) if len(r2_vals) else np.nan
        sd_r2 = float(np.std(r2_vals, ddof=1)) if len(r2_vals) > 1 else np.nan
        row = {"Model": m, "Mean_R2": mean_r2, "SD_R2": sd_r2, "Group": cld.get(m,"")}
        row.update(boot_df.loc[boot_df["model"]==m].drop(columns=["model"]).to_dict(orient="records")[0])
        summary_rows.append(row)
    summary_df = pd.DataFrame(summary_rows)

    # Save
    ci_tag = f"ci{int(ci_level*100)}"
    OUT = "statsig_outputs"
    os.makedirs(OUT, exist_ok=True)
    shapiro_df.to_csv(os.path.join(OUT, f"shapiro_{ci_tag}.csv"), index=False)
    levene_df.to_csv(os.path.join(OUT, f"levene_{ci_tag}.csv"), index=False)
    anova_out.to_csv(os.path.join(OUT, f"anova_r2_{ci_tag}.csv"))
    summary_df.to_csv(os.path.join(OUT, f"summary_r2_mse_{ci_tag}.csv"), index=False)

    return {"summary": summary_df, "anova": anova_out, "shapiro": shapiro_df, "levene": levene_df, "raw": df, "tukey": tukey.summary()}


In [ ]:

def plot_r2_bar_with_groups(summary_df: pd.DataFrame, ci_level: float = CI_LEVEL, save=True):
    labels = summary_df["Model"].tolist()
    means = summary_df["Mean_R2"].values
    groups = summary_df["Group"].tolist()

    plt.figure(figsize=(9,5))
    plt.bar(labels, means)
    for i, g in enumerate(groups):
        plt.text(i, means[i] + 0.003, g, ha="center", va="bottom", fontsize=10)
    plt.ylabel("Mean R2")
    plt.title("Mean R2 by Model (Tukey CLD)")
    plt.xticks(rotation=20)
    plt.tight_layout()
    if save:
        path = os.path.join("statsig_outputs", f"r2_bar_groups_ci{int(ci_level*100)}.png")
        plt.savefig(path, dpi=160)
        print("Saved:", path)
    plt.show()

def plot_mse_bootstrap_hist(df_metrics: pd.DataFrame, ci_level: float = CI_LEVEL, bootstrap_n: int = BOOTSTRAP_N, save=True):
    rng = np.random.default_rng(42)
    for m in MODEL_NAMES:
        mse_vals = df_metrics.loc[df_metrics["model"] == m, "MSE"].dropna().values
        if len(mse_vals) == 0:
            continue
        bs_means = [np.mean(rng.choice(mse_vals, size=len(mse_vals), replace=True))
                    for _ in range(bootstrap_n)]
        plt.figure(figsize=(8,4))
        plt.hist(bs_means, bins=40)
        plt.title(f"Bootstrap Mean MSE — {m}")
        plt.xlabel("Mean MSE")
        plt.ylabel("Frequency")
        plt.tight_layout()
        if save:
            p = os.path.join("statsig_outputs", f"bootstrap_mse_{m.replace('+','_')}_ci{int(ci_level*100)}.png")
            plt.savefig(p, dpi=160)
            print("Saved:", p)
        plt.show()



## Run
- Set `METRICS_CSV` to your CSV path and execute.
- The notebook will generate tables under `statsig_outputs/` and figures for quick inspection.


In [ ]:

if METRICS_CSV:
    df_metrics = load_metrics_from_csv(METRICS_CSV)
    print("Loaded:", METRICS_CSV)
    out = run_statsig_pipeline(df_metrics, ci_level=CI_LEVEL, bootstrap_n=BOOTSTRAP_N)
    display(out["summary"])
    display(out["anova"])
    display(out["shapiro"])
    display(out["levene"])
    plot_r2_bar_with_groups(out["summary"], ci_level=CI_LEVEL, save=True)
    plot_mse_bootstrap_hist(out["raw"], ci_level=CI_LEVEL, bootstrap_n=BOOTSTRAP_N, save=True)
else:
    print("Set METRICS_CSV to your per-fold results CSV and re-run this cell.")
